In [ ]:
import os
import joblib
from pathlib import Path
from matplotlib import colormaps
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import random
import typer

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer, label_binarize
from sklearn.decomposition import PCA

from skimage import exposure
from skimage.color import rgb2gray
from skimage.feature import hog, local_binary_pattern, corner_harris, corner_subpix, corner_peaks

from scipy.ndimage import gaussian_filter, gaussian_laplace, sobel
import cv2

### PATHS

In [ ]:
PROJ_ROOT = Path().resolve().parents[0]

DATA_DIR = PROJ_ROOT / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
EXTERNAL_DATA_DIR = DATA_DIR / "external"

MODELS_DIR = PROJ_ROOT / "models/tmp"

In [ ]:
input_path: Path = RAW_DATA_DIR / "cifar-10-batches-py"
data_path: Path = EXTERNAL_DATA_DIR / "processed_dataset.pkl"
predictions_path: Path = EXTERNAL_DATA_DIR / "test_predictions.csv"
model_path: Path = MODELS_DIR

### FEATURES AND DATA PROCESSING

In [ ]:
def load_data(file_path, encoding=None):
    """
    Load data from a pickle file

    Args:
        file_path: str
    """
    with open(file_path, 'rb') as f:
        if (encoding is not None):
            data = pickle.load(f, encoding=encoding)
        else:
            data = pickle.load(f)
    return data

def save_data(data, file):
    """
    Save data to a pickle file

    Args:
        data: any
        file: str
    """
    with open(file, 'wb') as f:
        pickle.dump(data, f)

def to_image(img_flat):
    """
    Convert a flattened image to a 3D image

    Args:
        img_flat: np.ndarray
    """
    img_R = img_flat[0:1024].reshape((32, 32))
    img_G = img_flat[1024:2048].reshape((32, 32))
    img_B = img_flat[2048:3072].reshape((32, 32))
    img = np.dstack((img_R, img_G, img_B))
    return img

def image_contrast(image):
    """
    Compute the contrast of an image

    Args:
        image: np.ndarray
    """
    lab= cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return enhanced_img

def extract_hog_features(images):
    """
    Extract HOG features from a list of images

    Args:
        images: np.ndarray
    """
    hog_features = []

    for image in images:
        gray_image = rgb2gray(image).astype(np.uint8)
        # hog_image_rescaled = exposure.rescale_intensity(gray_image, in_range=(0, 10))
        feature, hog_image = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, channel_axis=None, multichannel=False, block_norm='L2-Hys')
        # feature, hog_image = hog(hog_image_rescaled, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, channel_axis=None, multichannel=False, block_norm='L2-Hys')
        hog_features.append(feature)

    return np.array(hog_features)

def extract_sift_features(images):
    """
    Extract SIFT features from a list of images

    Args:
        images: np.ndarray
    """
    sift = cv2.SIFT_create()
    sift_features = []

    for image in images:
        gray_image = rgb2gray(image).astype(np.uint8)
        # print(image.shape, gray_image.shape, image.dtype, gray_image.dtype)
        keypoints, descriptors = sift.detectAndCompute(gray_image, None)
        if descriptors is None:
            sift_features.append(np.zeros((128,)))
        else:
            sift_features.append(descriptors.flatten())
    
    return np.array(sift_features)

### VLAD

In [ ]:
from random import sample

import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import progressbar as pb


class VLAD:
    """VLAD - Vector of Locally Aggregated Descriptors

    This class provides an implementation of the original proposal of "Vector of Locally Aggregated Descriptors" (VLAD)
    originally proposed in [1]_.

    Parameters
    ----------
    k : int, default=256
        Number of clusters to obtain for the visual vocabulary.
    n_vocabs : int, default=1
        Number of vocabularies to use
    norming : {"original", "intra", "RN"}, default="original"
        How the norming of the VLAD-descriptors should be performed.
        For more info see below.
    lcs : bool, default=True
        If `True`, uses Local Coordinate System (LCS) described in [3]_.
    alpha : float, default=0.2
        The exponent for the root-part, default taken from [3]_
    verbose : bool, default=True
        If `True` print messages here and there

    Attributes
    ----------
    vocabs : sklearn.cluster.KMeans(k)
        The visual vocabulary of the object
    centers : array
        The centroids for the visual vocabulary
    database : array
        All known VLAD-vectors

    Notes
    -----
    ``norming="original"`` uses the original formulation of [1]_. An updated formulation based on [2]_
    is provided by ``norming="intra"``. Finally the best norming based on [3]_ is provided by ``norming="RN"``.

    References
    ----------
    .. [1] Jégou, H., Douze, M., Schmid, C., & Pérez, P. (2010, June). Aggregating
           local descriptors into a compact image representation. In 2010 IEEE computer
           society conference on computer vision and pattern recognition (pp. 3304-3311). IEEE.

    .. [2] Arandjelovic, R., & Zisserman, A. (2013). All about VLAD. In Proceedings of the
           IEEE conference on Computer Vision and Pattern Recognition (pp. 1578-1585).

    .. [3] Delhumeau, J., Gosselin, P. H., Jégou, H., & Pérez, P. (2013, October).
           Revisiting the VLAD image representation. In Proceedings of the 21st ACM
           international conference on Multimedia (pp. 653-656).

    .. [4] Jégou, H., & Chum, O. (2012, October). Negative evidences and co-occurences in image
           retrieval: The benefit of PCA and whitening. In European conference on computer vision
           (pp. 774-787). Springer, Berlin, Heidelberg.

    .. [5] Spyromitros-Xioufis, E., Papadopoulos, S., Kompatsiaris, I. Y., Tsoumakas, G., & Vlahavas,
           I. (2014). A comprehensive study over VLAD and product quantization in large-scale image retrieval.
           IEEE Transactions on Multimedia, 16(6), 1713-1728.
    """
    def __init__(self, k=256, n_vocabs=1, norming="original", lcs=False, alpha=0.2, verbose=True):
        """Initialize VLAD-object

        Notes
        -----

        Hyperparameters have to be set, even if `centers` and `qs` are set externally.
        """
        self.k = k
        self.n_vocabs = n_vocabs
        self.norming = norming
        self.vocabs = None
        self.centers = None
        self.database = None
        self.lcs = lcs
        self.alpha = alpha
        self.qs = None
        self.verbose = verbose

    def fit(self, X, y):
        """Fit Visual Vocabulary

        Parameters
        ----------
        X : list(array)
            List of image descriptors

        Returns
        -------
        self : VLAD
            Fitted object
        """
        X_mat = np.vstack(X)
        self.vocabs = []
        self.centers = []  # Is a list of `n_vocabs` np.arrays. Can be set externally without fitting
        self.qs = []  # Is a list of `n_vocabs` lists of length `k` of np.arrays. Can be set externally without fitting
        for i in range(self.n_vocabs):
            if self.verbose is True:
                print(f"Training vocab #{i+1}")
            if self.verbose is True:
                print(f"Training KMeans...")
            if len(X_mat) < int(2e5):
                self.vocabs.append(KMeans(n_clusters=self.k).fit(X_mat))
            else:
                idx = sample(range(len(X_mat)), int(2e5))
                self.vocabs.append(KMeans(n_clusters=self.k).fit(X_mat[idx]))
            self.centers.append(self.vocabs[i].cluster_centers_)
            if self.lcs is True and self.norming == "RN":
                if self.verbose is True:
                    print("Finding rotation-matrices...")
                predicted = self.vocabs[i].predict(X_mat)
                qsi = []
                for j in range(self.k):
                    q = PCA(n_components=X_mat.shape[1]).fit(X_mat[predicted == j]).components_
                    qsi.append(q)
                self.qs.append(qsi)
        self.database = self._extract_vlads(X)
        return self

    def transform(self, X):
        """Transform the input-tensor to a matrix of VLAD-descriptors

        Parameters
        ----------
        X : list(array)
            List of image-descriptors

        Returns
        -------
        vlads : array, shape (n, d * self.k)
            The transformed VLAD-descriptors
        """
        vlads = self._extract_vlads(X)
        return vlads

    def fit_transform(self, X, y):
        """Fit the model and transform the input-data subsequently

        Parameters
        ----------
        X : list(array)
            List of image-descriptors

        Returns
        -------
        vlads : array, shape (n, d * self.k)
            The transformed VLAD-descriptors
        """
        _ = self.fit(X, y)
        vlads = self.transform(X)
        return vlads

    def refit(self, X):
        """Refit the Visual Vocabulary

        Uses the already learned cluster-centers as in initial values for
        the KMeans-models

        Parameters
        ----------
        X : array
            The database used to refit the visual vocabulary

        Returns
        -------
        self : VLAD
            Refitted object
        """
        self.vocabs = []
        self.centers = []

        for i in range(self.n_vocabs):
            self.vocabs.append(KMeans(n_clusters=self.k, init=self.centers).fit(X.transpose((2, 0, 1))
                                                                                .reshape(-1, X.shape[1])))
            self.centers.append(self.vocabs[i].cluster_centers_)

        self.database = self._extract_vlads(X)
        return self

    def predict(self, desc):
        """Predict class of given descriptor-matrix

        Parameters
        ----------
        desc : array
            A descriptor-matrix (m x d)

        Returns
        -------
        ``argmax(self.predict_proba(desc))`` : array
        """
        return np.argmax(self.predict_proba(desc))

    def predict_proba(self, desc):
        """Predict class of given descriptor-matrix, return probability

        Parameters
        ----------
        desc : array
            A descriptor-matrix (m x d)

        Returns
        -------
        ``self.database @ vlad``
            The similarity for all database-classes
        """
        vlad = self._vlad(desc)  # Convert to VLAD-descriptor
        return self.database @ vlad  # Similarity between L2-normed vectors is defined as dot-product

    def _vlad(self, X):
        """Construct the actual VLAD-descriptor from a matrix of local descriptors

        Parameters
        ----------
        X : array
            Descriptor-matrix for a given image

        Returns
        -------
        ``V.flatten()`` : array
            The VLAD-descriptor
        """
        np.seterr(invalid='ignore', divide='ignore')  # Division with 0 encountered below
        vlads = []

        for j in range(self.n_vocabs):  # Compute for multiple vocabs
            # predicted = self.vocabs[j].predict(X)  # Commented out in favor of line below (No dependency on actual vocab, but only on centroids)
            predicted = norm(X - self.centers[j][:, None, :], axis=-1).argmin(axis=0)
            _, d = X.shape
            V = np.zeros((self.k, d))  # Initialize VLAD-Matrix

            # Computing residuals
            if self.norming == "RN":
                curr = X - self.centers[j][predicted]
                curr /= norm(curr, axis=1)[:, None]
                # Untenstehendes kann noch vektorisiert werden

                for i in range(self.k):
                    V[i] = np.sum(curr[predicted == i], axis=0)
                    if self.lcs is True:
                        V[i] = self.qs[j][i] @ V[i]  # Equivalent to multiplication in  summation above
            else:
                for i in range(self.k):
                    V[i] = np.sum(X[predicted == i] - self.centers[j][i], axis=0)

            # Norming
            if self.norming in ("intra", "RN"):
                V /= norm(V, axis=1)[:, None]  # L2-normalize every sum of residuals
                np.nan_to_num(V, copy=False)  # Some of the rows contain 0s. np.nan will be inserted when dividing by 0!

            if self.norming in ("original", "RN"):
                V = self._power_law_norm(V)

            V /= norm(V)  # Last L2-norming
            V = V.flatten()
            vlads.append(V)
        vlads = np.concatenate(vlads)
        vlads /= norm(vlads)  # Not on axis, because already flat
        return vlads

    def _extract_vlads(self, X):
        """Extract VLAD-descriptors for a number of images

        Parameters
        ----------
        X : list(array)
            List of image-descriptors

        Returns
        -------
        database : array
            Database of all VLAD-descriptors for the given Tensor
        """
        vlads = []
        if self.verbose:
            for x in pb.progressbar(X):
                vlads.append(self._vlad(x))
        else:
            for x in X:
                vlads.append(self._vlad(x))

        database = np.vstack(vlads)
        return database

    def _add_to_database(self, vlad):
        """Add a given VLAD-descriptor to the database

        Parameters
        ----------
        vlad : array
            The VLAD-descriptor that should be added to the database

        Returns
        -------
        ``None``
        """
        self.database = np.vstack((self.database, vlad))

    def _power_law_norm(self, X):
        """Perform power-Normalization on a given array

        Parameters
        ----------
        X : array
            Array that should be normalized

        Returns
        -------
        normed : array
            Power-normalized array
        """
        normed = np.sign(X) * np.abs(X)**self.alpha
        return normed

    def __repr__(self):
        return f"VLAD(k={self.k}, norming=\"{self.norming}\")"

    def __str__(self):
        return f"VLAD(k={self.k}, norming=\"{self.norming}\")"

### METHODS FOR MODELS LOADING & SAVING

In [ ]:
def save_model(model, model_name, model_dir='models'):
    """
    Save the trained model to a file.
    
    Args:
        model: Trained model to be saved.
        model_name: The name of the model file.
        model_dir: Directory where the model will be saved.
    """
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    file_path = os.path.join(model_dir, model_name)
    joblib.dump(model, file_path)
    print(f"Model saved to {file_path}")

def load_model(model_name, model_dir='models'):
    """
    Load a trained model from a file.
    
    Args:
        model_name: The name of the model file.
        model_dir: Directory where the model is saved.
    
    Returns:
        Loaded model.
    """
    file_path = os.path.join(model_dir, model_name)
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"No model found at {file_path}")
    
    model = joblib.load(file_path)
    print(f"Model loaded from {file_path}")
    return model

### DATASET

In [ ]:
print("Processing dataset...")
meta_dict = load_data(input_path / "batches.meta", encoding='bytes')
label_names = meta_dict[b'label_names']
print(f"Labels/Features: {label_names}")

combined_dict = {
    b'data': [],
    b'labels': [],
    b'label_names': label_names,
    b'hog_data': [],
    b'validation_data': [],
    b'validation_labels': [],
    b'validation_hog_data': [],
    b'test_data': [],
    b'test_labels': [],
    b'test_hog_data': [],
    b'sample_images': {label: [] for label in label_names}
}

for i in range(5):
    batch_dict = load_data(input_path / f"data_batch_{i+1}", encoding='bytes')
    combined_dict[b'data'].extend(batch_dict[b'data'])
    combined_dict[b'labels'].extend(batch_dict[b'labels'])

# Diviser les données en données d'entraînement et de validation
combined_dict[b'data'], combined_dict[b'validation_data'], combined_dict[b'labels'], combined_dict[b'validation_labels'] = train_test_split(
    combined_dict[b'data'], combined_dict[b'labels'], test_size=0.2, random_state=42
)

# Charger les données de test depuis test_batch
test_batch_dict = load_data(input_path / "test_batch", encoding='bytes')
combined_dict[b'test_data'].extend(test_batch_dict[b'data'])
combined_dict[b'test_labels'].extend(test_batch_dict[b'labels'])

# Sélectionner 10 images aléatoires pour chaque classe
for label_index, label in enumerate(label_names):
    label_indices = [i for i, lbl in enumerate(combined_dict[b'labels']) if lbl == label_index]
    sample_indices = random.sample(label_indices, 10)
    for idx in sample_indices:
        combined_dict[b'sample_images'][label].append(combined_dict[b'data'][idx])

save_data(combined_dict, data_path)

### DATA DICTIONARY KEYS

In [ ]:
combined_dict.keys()

### MODELS

In [ ]:
models = {
    'random_forest': RandomForestClassifier(random_state=42, criterion='entropy', max_depth=10), # criterion, max_depth
    'svm': SVC(max_iter=100 , kernel='rbf', random_state=42), # degree, gamma
    'logistic_regression': LogisticRegression(random_state=42), # penalty, solver
    'knn': KNeighborsClassifier(), # n_neighbors, algorithm
    # 'gradient_boosting': GradientBoostingClassifier(learning_rate=0.01, random_state=42) # loss, max_depth, criterion
}

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class ImageFlattener(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.reshape(X.shape[0], -1)

### PIPELINE

In [ ]:
pipeline = Pipeline([
    ('grayscale', FunctionTransformer(rgb2gray, validate=False)),
    # ('flatten', ImageFlattener()),
    # ('sift_extraction', FunctionTransformer(extract_sift_features, validate=False)),
    # ('hog_extraction', FunctionTransformer(extract_hog_features, validate=False)),
    ('vlad', VLAD(k=16, norming="RN")),
    # ('scaler', StandardScaler()),
    # ('pca', PCA(n_components=10)),
    ('classifier', None)
])

### TRAINING & EVALUATING

In [ ]:
data = load_data(data_path)

In [ ]:
X_train = np.array(data[b'data']).reshape(-1, 32, 32, 3) / 255.0
X_train = X_train.reshape(X_train.shape[0], -1)
y_train = np.array(data[b'labels'])
X_train = np.array([to_image(img) for img in X_train])


train_dict = {
    b'data': X_train,
    b'labels': y_train,
}
save_data(train_dict, EXTERNAL_DATA_DIR / "train_data.pkl")

X_validation = np.array(data[b'validation_data']).reshape(-1, 32, 32, 3) / 255.0
X_validation = X_validation.reshape(X_validation.shape[0], -1)
y_validation = np.array(data[b'validation_labels'])
X_validation = np.array([to_image(img) for img in X_validation])


validation_dict = {
    b'data': X_validation,
    b'labels': y_validation,
}
save_data(validation_dict, EXTERNAL_DATA_DIR / "validation_data.pkl")

# Train the models
for name, model in models.items():
    print(f"Training {name} model...")
    pipeline.set_params(classifier=model)

    pipeline.fit(X_train, y_train)
    print(f"{name} model trained.")

    y_pred = pipeline.predict(X_validation)

    accuracy = accuracy_score(y_validation, y_pred)
    print(f"{name} model accuracy: {accuracy:.2f}")

    save_model(model, f"{name}_model.pkl", model_path)
    print(f"{name} model saved.")

### TESTING

In [ ]:
X_test = np.array(data[b'test_data']).reshape(-1, 32, 32, 3) / 255.0
X_test = X_test.reshape(X_test.shape[0], -1)
X_test_2d = X_test
y_test = np.array(data[b'test_labels'])
X_test = np.array([to_image(img) for img in X_test])

test_dict = {
    b'data': X_test,
    b'labels': y_test,
}
save_data(test_dict, EXTERNAL_DATA_DIR / "test_data.pkl")

predictions = {'True Labels': y_test}

# Evaluate the models
for name, _ in models.items():
    model = load_model(f'{name}_model.pkl', model_path)
    print(f"Evaluating {name} model...")

    test_pipeline = Pipeline([
        ('greyscale', FunctionTransformer(rgb2gray, validate=False)),
        # ('flatten', ImageFlattener()),
        # ('sift_extraction', FunctionTransformer(extract_sift_features, validate=False)),
        # ('hog_extraction', FunctionTransformer(extract_hog_features, validate=False)),
        ('vlad', VLAD(k=16, norming="RN")),
        # ('scaler', StandardScaler()),
        # ('pca', PCA(n_components=10)),
        ('classifier', model)
    ])

    y_pred = test_pipeline.predict(X_test)
    predictions[name] = y_pred

    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} model accuracy: {accuracy:.2f}")
    
    scores = cross_val_score(model, X_test_2d, y_test, cv=5)
    print(f"{name} model cross-validation accuracy: {scores.mean():.2f}")

predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv(predictions_path, index=False)
print(f"Predictions saved to {predictions_path}")

### GRAPH PLOTS

#### Plot Histogram of Colors feature

In [ ]:
def plot_color_histo_features(image, save_path=None):
    gray_image = rgb2gray(image)

    _, axis = plt.subplots(2, 2, gridspec_kw={'width_ratios': [1, 3]})
    
    # Grascale Image
    axis[1][0].imshow(gray_image, cmap='gray')
    axis[1][1].set_title('Histogram')
    axis[1][0].set_title('Grayscale Image')
    axis[1][0].axis('off')
    hist = exposure.histogram(gray_image)
    axis[1][1].plot(hist[0])

    # Color image
    if image.ndim == 3:
        axis[0][0].imshow(image, cmap='gray')
        axis[0][1].set_title('Histogram')
        axis[0][0].set_title('Original Image')
        axis[0][0].axis('off')
        rgbcolors = ['red', 'green', 'blue']
        for i, mycolor in enumerate(rgbcolors):
            axis[0][1].plot(exposure.histogram(image[...,i])[0], color=mycolor)
    
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()

sample_image = X_train[np.random.randint(0, X_train.shape[0])]
plot_color_histo_features(sample_image, "../reports/color_histo_features.png")

# sample_image = X_train[0]
# plot_color_histo_features(sample_image)

#### Plot HOG feature

In [ ]:
def plot_hog_features(image, save_path=None):
    """
    Plot the HOG features of an image.

    Args:
        image: The image to plot the HOG features.
        save_path: The path to save the plot.
    """
    # Convertir l'image en niveaux de gris
    gray_image = rgb2gray(image)

    # Extraire les caractéristiques HOG de l'image
    feature, hog_image = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, channel_axis=None, block_norm='L2-Hys')
    
    # Normaliser l'image HOG
    # hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10)) # L'image est déjà normalisée au préalable
    
    # Afficher l'image originale et l'image HOG
    fig, axs = plt.subplots(1, 3, figsize=(10, 5))
    axs[0].imshow(image)
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    axs[1].imshow(hog_image)
    axs[1].set_title('HOG Image')
    axs[1].axis('off')

    axs[2].imshow(hog_image, cmap='gray')
    axs[2].set_title('HOG Image with grayscale')
    axs[2].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()

sample_image = X_train[np.random.randint(0, X_train.shape[0])]
plot_hog_features(sample_image, "../reports/hog_features.png")

# sample_image = X_train[0]
# plot_hog_features(sample_image)

#### Plot SIFT features

In [ ]:
def plot_sift_features(image, save_path=None):
    """
    Plots the original image, grayscale image, keypoints on grayscale image, and final image with keypoints overlaid.
    
    Args:
        image: The input image as a numpy array.
        save_path: The path to save the plot.
    """
    # Convertir l'image en niveaux de gris
    gray_image = (rgb2gray(image) * 255).astype(np.uint8)
    image_8bit = (image * 255).astype(np.uint8)

    # Initialisation du détecteur de points clés SIFT
    sift = cv2.SIFT_create()
    
    # Détection des points clés et calcul des descripteurs
    keypoints, descriptors = sift.detectAndCompute(gray_image, None)

    # Afficher les points clés sur l'image originale
    image_with_keypoints = cv2.drawKeypoints(image_8bit, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    # Afficher les points clés sur l'image en niveaux de gris
    gray_image_with_keypoints = cv2.drawKeypoints(gray_image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    # Afficher les images: originale, en niveaux de gris, en niveaux de gris avec les points clés, et originale avec les points clés
    fig, axs = plt.subplots(2, 2, figsize=(10, 8))
    
    axs[0][0].imshow(image)
    axs[0][0].set_title('Original Image')
    axs[0][0].axis('off')

    axs[0][1].imshow(gray_image, cmap='gray')
    axs[0][1].set_title('Grayscale Image')
    axs[0][1].axis('off')
        
    axs[1][0].imshow(image_with_keypoints)
    axs[1][0].set_title('Image with SIFT Keypoints')
    axs[1][0].axis('off')
                   
    axs[1][1].imshow(gray_image_with_keypoints)
    axs[1][1].set_title('Grayscale Image with SIFT Keypoints')
    axs[1][1].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()

sample_image = X_train[np.random.randint(0, X_train.shape[0])]
plot_sift_features(sample_image, "../reports/sift_features.png")

# sample_image = X_train[0]
# plot_sift_features(sample_image)

#### Corner detection

In [ ]:
def plot_corner_detect_features(image, save_path=None):
    """
    Plot the corner detection features of an image.

    Args:
        image: The image to plot the corner detection features.
        save_path: The path to save the plot.
    """
    
    gray_image = rgb2gray(image)
    image = (image * 255).astype(np.uint8)

    coords = corner_peaks(corner_harris(gray_image), min_distance=2, threshold_rel=0.01)
    coords_subpix = corner_subpix(gray_image, coords, window_size=4)

    fig, axis = plt.subplots(2, 2, figsize=(6, 6))
    axis[0][0].axis('off')
    axis[0][0].imshow(image)
    axis[0][0].plot(
        coords[:, 1], coords[:, 0], color='cyan', marker='o', linestyle='None', markersize=6
    )
    axis[0][0].plot(coords_subpix[:, 1], coords_subpix[:, 0], '+r', markersize=15)

    axis[0][1].axis('off')
    axis[0][1].imshow(gray_image, cmap='gray')
    axis[0][1].plot(
        coords[:, 1], coords[:, 0], color='cyan', marker='o', linestyle='None', markersize=6
    )
    axis[0][1].plot(coords_subpix[:, 1], coords_subpix[:, 0], '+r', markersize=15)
    
    # ----------- Contrasted Image ------------
    contrasted_image = image_contrast(image)
    gray_contrasted_image = rgb2gray(contrasted_image)

    coords2 = corner_peaks(corner_harris(gray_image), min_distance=2, threshold_rel=0.01)
    coords_subpix2 = corner_subpix(gray_image, coords2, window_size=4)

    axis[1][0].axis('off')
    axis[1][0].imshow(contrasted_image)
    axis[1][0].plot(
        coords2[:, 1], coords2[:, 0], color='cyan', marker='o', linestyle='None', markersize=6
    )
    axis[1][0].plot(coords_subpix2[:, 1], coords_subpix2[:, 0], '+r', markersize=15)

    axis[1][1].axis('off')
    axis[1][1].imshow(gray_contrasted_image, cmap='gray')
    axis[1][1].plot(
        coords2[:, 1], coords2[:, 0], color='cyan', marker='o', linestyle='None', markersize=6
    )
    axis[1][1].plot(coords_subpix2[:, 1], coords_subpix2[:, 0], '+r', markersize=15)
    

    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()

sample_image = X_train[np.random.randint(0, X_train.shape[0])]
plot_corner_detect_features(sample_image, "../reports/corner_detect_features.png")

# sample_image = X_train[0]
# plot_corner_detect_features(sample_image)

In [ ]:
predictions_df = pd.read_csv(predictions_path)
data = load_data(data_path)
label_names = [label.decode('utf-8') for label in data[b'label_names']]

train_data = load_data(EXTERNAL_DATA_DIR / "train_data.pkl")
test_data = load_data(EXTERNAL_DATA_DIR / "test_data.pkl")

X_train = np.array(train_data[b'data'])
y_train = np.array(train_data[b'labels'])
X_test = np.array(test_data[b'data'])
y_test = np.array(test_data[b'labels'])

#### Correlations, Matrices de Confusion, Courbe Precision-Recall, Courbes ROC

In [ ]:
# Correlation des prédictions
correlation_matrix = predictions_df.corr()
plt.figure(figsize=(10, 7))
plt.title('Correlation Matrix of Model Predictions')
sns.heatmap(correlation_matrix, annot=True, cmap='plasma')
plt.show()

n_classes = len(label_names)
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

# Matrice de confusion
for name, _ in models.items():
    print("=============================================================")
    print(f"Plotting evaluation metrics for {name}...")
    predictions = predictions_df[name]
    cmat = confusion_matrix(y_test, predictions)
    plt.figure(figsize=(10, 7))
    plt.title(f'Confusion Matrix for {name}')
    plt.xticks(rotation=45)
    sns.heatmap(cmat, annot=True, fmt='d', cmap='Blues', xticklabels=label_names, yticklabels=label_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

    # Courbes de précision/rappel et ROC
    preds_bin = label_binarize(predictions, classes=np.arange(n_classes))
    
    # Precision-Recall curve
    plt.figure()
    for i, label in enumerate(label_names):
        precision, recall, _ = precision_recall_curve(y_test_bin[:, i], preds_bin[:, i])
        plt.plot(recall, precision, lw=2, label=f'{label}')
    plt.xticks(rotation=45)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall curve for {name}')
    plt.legend(loc='best')
    plt.show()
    
    # ROC curve
    plt.figure()
    for i, label in enumerate(label_names):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], preds_bin[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{label} (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xticks(rotation=45)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC curve for {name}')
    plt.legend(loc='best')
    plt.show()

### PLOT CLASSIFIER COMPARISON

In [ ]:
X = np.append(X_train, X_test, axis=0)
y = np.append(y_train, y_test)

figure = plt.figure(figsize=(27, 9))
i = 1

x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

cm = colormaps.get_cmap('tab10')
cm_bright = ListedColormap(["red", "blue", "green", "purple", "orange", "brown", "pink", "gray", "olive", "cyan"])

ax = plt.subplot(1, len(models) + 1, i)
ax = plt.subplot(len(data), len(models) + 1, i)

ax.set_title('Input data')
ax.scatter(X_train[:,0], X_train[:,1], c=[(1,0,0)]*len(y_train), cmap=cm_bright, edgecolors='k')
ax.scatter(X_test[:,0], X_test[:,1], c=y_test, cmap=cm_bright, edgecolors='k', alpha=0.6)
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_xticks(())
ax.set_yticks(())
i += 1

for name, _ in models.items():
    model = joblib.load(model_path / f'{name}_model.pkl')
    score = model.score(X_test, y_test)

    ax = plt.subplot(1, len(models) + 1, i)

    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, edgecolors='k')
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, edgecolors='k', alpha=0.6)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(f'{name} (score = {score:.2f})')
    ax.text(x_max - 0.3, y_min + 0.3, ("%.2f" % score).lstrip("0"),
            size=15, horizontalalignment='right')
    i += 1

plt.tight_layout()
plt.show()